# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Необходимо:
- обучить модель классифицировать комментарии на позитивные и негативные. 
  <br>В нашем распоряжении набор данных с разметкой о токсичности правок.
- построить модель со значением метрики качества *F1* не меньше 0.75. 

**План работы**

1. Загрузка и подготовка данных.
2. Обучение разных моделей. 
3. Формулирование выводов.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. 
<br>Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

Импорт необходимых для исследования инструментов

In [ ]:
import pandas as pd
import numpy as np
import nltk
import time
import re
from tqdm import notebook
from nltk.corpus import stopwords as nltk_stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.dummy import DummyClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import make_scorer
from sklearn.metrics import f1_score
from catboost import CatBoostClassifier, Pool

### Знакомство с данными

Открытие файла и знакомство с данными

In [ ]:
df = pd.read_csv('/datasets/toxic_comments.csv', index_col=0)
display(df.head())
df.info()

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


<class 'pandas.core.frame.DataFrame'>
Int64Index: 159292 entries, 0 to 159450
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159292 non-null  object
 1   toxic   159292 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 3.6+ MB


Проверка на наличие явных дубликатов строк

In [ ]:
df.duplicated().sum()

0

Проверка количества позитивных и негативных комментариев

In [ ]:
df['toxic'].value_counts(normalize=True)

0    0.898388
1    0.101612
Name: toxic, dtype: float64

### Предобработка данных

Функция для лемматизации, удаления цифр, символов и лишних пробелов

In [ ]:
def text_preprocessing(text):
    tokenized = nltk.word_tokenize(text)
    joined = ' '.join(tokenized)
    text_only = re.sub(r'[^a-zA-Z]', ' ', joined)
    desired_result = ' '.join(text_only.split())
    return desired_result

Применение функции к исследуемому тексту

In [ ]:
notebook.tqdm.pandas() 
df['text_clear'] = df['text'].progress_apply(text_preprocessing)

  0%|          | 0/159292 [00:00<?, ?it/s]

Деление датафрейма на две выборки: обучающую (75%) и тестовую (25%)

In [ ]:
X_train_clear, X_test_clear = train_test_split(df['text_clear'], test_size=.25, random_state=12345)
y_train, y_test = train_test_split(df['toxic'], test_size=.25, random_state=12345)

X_train_clear.shape, X_test_clear.shape, y_train.shape, y_test.shape

((119469,), (39823,), (119469,), (39823,))

Очистка текста от стоп-слов

In [ ]:
%time
stopwords = set(nltk_stopwords.words('english'))
count_tf_idf = TfidfVectorizer(stop_words=stopwords)

X_train = count_tf_idf.fit_transform(X_train_clear)
X_test = count_tf_idf.transform(X_test_clear)
X_train.shape, X_test.shape

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 5.72 µs


((119469, 142013), (39823, 142013))

### Вывод

- данные импортированы с удалением лишнего индекса при чтении
- удалены стоп-слова
- явных дубликатов не обнаружено
- в данных наблюдается большой дисбаланс классов
- данные поделены на две выборки: обучающую и тестовую

## Обучение

Выделение констант для обучения моделей

In [ ]:
N_SPLITS = 3
RS = [12345]

Создание вызываемого объекта для оценки

In [ ]:
score = make_scorer(f1_score)

### Модель *LogisticRegression*

Кросс-валидация  и обучение модели *Логистической регрессии*

In [ ]:
%%time

params_lr = {"class_weight": ['balanced'], 
             "random_state": RS,
             "max_iter": [100, 150, 200],
             "C": [0.5, 1.0, 1.5]
            }
model_lr = LogisticRegression()
model_lr_rs = RandomizedSearchCV(
    model_lr, cv=N_SPLITS, scoring=score, 
    param_distributions=params_lr, n_iter=N_SPLITS, verbose=True
)
model_lr_rs.fit(X_train, y_train)

Fitting 3 folds for each of 3 candidates, totalling 9 fits
CPU times: user 2min 36s, sys: 3min 4s, total: 5min 40s
Wall time: 5min 40s


RandomizedSearchCV(cv=3, estimator=LogisticRegression(), n_iter=3,
                   param_distributions={'C': [0.5, 1.0, 1.5],
                                        'class_weight': ['balanced'],
                                        'max_iter': [100, 150, 200],
                                        'random_state': [12345]},
                   scoring=make_scorer(f1_score), verbose=True)

Вывод лучших параметров

In [ ]:
display('Лучший результат f1-меры:', model_lr_rs.best_score_)
model_lr_rs.best_params_

'Лучший результат f1-меры:'

0.7529015458098378

{'random_state': 12345, 'max_iter': 200, 'class_weight': 'balanced', 'C': 1.5}

### Модель *RandomForestClassifier*

Обучение модели *Случайного Леса*

In [ ]:
%%time

params = {"max_depth": [5, 10, 15], 
          "n_estimators": [30, 90, 150], 
          "random_state": RS, 
          "class_weight": ['balanced']
         }
model_rfc = RandomForestClassifier()

model_rfc_rs = RandomizedSearchCV(model_rfc, cv=N_SPLITS, scoring=score, param_distributions=params)
model_rfc_rs.fit(X_train, y_train)

/opt/conda/lib/python3.9/site-packages/sklearn/model_selection/_search.py:285: UserWarning: The total space of parameters 9 is smaller than n_iter=10. Running 9 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


CPU times: user 6min 6s, sys: 0 ns, total: 6min 6s
Wall time: 6min 6s


RandomizedSearchCV(cv=3, estimator=RandomForestClassifier(),
                   param_distributions={'class_weight': ['balanced'],
                                        'max_depth': [5, 10, 15],
                                        'n_estimators': [30, 90, 150],
                                        'random_state': [12345]},
                   scoring=make_scorer(f1_score))

Вывод лучших параметров

In [ ]:
display('Лучший результат f1-меры:', model_rfc_rs.best_score_)
model_rfc_rs.best_params_

'Лучший результат f1-меры:'

0.41332566634279905

{'random_state': 12345,
 'n_estimators': 90,
 'max_depth': 15,
 'class_weight': 'balanced'}

### Модель *CatBoostClassifier*

Обучение модели *CatBoost*

In [ ]:
%%time

params = {"max_depth": [1, 2, 3], 
          "n_estimators": [10, 100, 150], 
          "random_state": RS, 
          "auto_class_weights": ['Balanced']
         }
model_cbc = CatBoostClassifier(eval_metric='TotalF1')

model_cbc_gs = model_cbc.grid_search(params, Pool(X_train,y_train),verbose=True, cv=N_SPLITS, plot=True)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 0.4865936	test: 0.4841108	best: 0.4841108 (0)	total: 561ms	remaining: 5.05s
1:	learn: 0.4865936	test: 0.4841108	best: 0.4841108 (0)	total: 1.04s	remaining: 4.15s
2:	learn: 0.4865936	test: 0.4841108	best: 0.4841108 (0)	total: 1.53s	remaining: 3.56s
3:	learn: 0.4865936	test: 0.4841108	best: 0.4841108 (0)	total: 2.02s	remaining: 3.03s
4:	learn: 0.5451321	test: 0.5396663	best: 0.5396663 (4)	total: 2.5s	remaining: 2.5s
5:	learn: 0.5451321	test: 0.5396663	best: 0.5396663 (4)	total: 2.97s	remaining: 1.98s
6:	learn: 0.5451321	test: 0.5396663	best: 0.5396663 (4)	total: 3.45s	remaining: 1.48s
7:	learn: 0.5451321	test: 0.5396663	best: 0.5396663 (4)	total: 3.92s	remaining: 981ms
8:	learn: 0.5467378	test: 0.5415895	best: 0.5415895 (8)	total: 4.41s	remaining: 490ms
9:	learn: 0.5467378	test: 0.5415895	best: 0.5415895 (8)	total: 4.89s	remaining: 0us

bestTest = 0.5415895333
bestIteration = 8

0:	loss: 0.5415895	best: 0.5415895 (0)	total: 58.6s	remaining: 7m 48s
0:	learn: 0.4865936	test: 0.48

Вывод лучших параметров

In [ ]:
display('Лучший результат f1-меры:', model_cbc.best_score_)
model_cbc.get_all_params()

'Лучший результат f1-меры:'

{'learn': {'Logloss': 0.26642270222050074, 'TotalF1': 0.9048847996908846}}

{'nan_mode': 'Min',
 'eval_metric': 'TotalF1',
 'iterations': 150,
 'sampling_frequency': 'PerTree',
 'leaf_estimation_method': 'Newton',
 'grow_policy': 'SymmetricTree',
 'penalties_coefficient': 1,
 'boosting_type': 'Plain',
 'model_shrink_mode': 'Constant',
 'feature_border_type': 'GreedyLogSum',
 'bayesian_matrix_reg': 0.10000000149011612,
 'force_unit_auto_pair_weights': False,
 'l2_leaf_reg': 3,
 'random_strength': 1,
 'rsm': 1,
 'boost_from_average': False,
 'model_size_reg': 0.5,
 'pool_metainfo_options': {'tags': {}},
 'subsample': 0.800000011920929,
 'use_best_model': False,
 'class_names': [0, 1],
 'random_seed': 12345,
 'depth': 3,
 'posterior_sampling': False,
 'border_count': 254,
 'class_weights': [1, 8.83931827545166],
 'classes_count': 0,
 'auto_class_weights': 'Balanced',
 'sparse_features_conflict_fraction': 0,
 'leaf_estimation_backtracking': 'AnyImprovement',
 'best_model_min_trees': 1,
 'model_shrink_rate': 0,
 'min_data_in_leaf': 1,
 'loss_function': 'Logloss',
 

### Вывод

- При обучении *Логистической Регрессии* получено значение *f1=0.753* за 5min 37s обучения
- При обучении *Случайного Леса* получено значение *f1=0.41* за 5min 6s обучения
- При обучении *CatBoost* получено значение *f1=0.9* за 18min 20s обучения
<br>С заданием справились две модели, но обучение ***LogisticRegression*** проходит значительно быстрее

## Тестирование выбранной модели

In [ ]:
predict = model_lr_rs.predict(X_test)
f1_score(y_test, predict)

0.7585751978891822

### Проверка адекватности модели

Сравнение результатов выбранной модели с результатами простейшей модели для проверки адекватности предсказаний

In [ ]:
dummy_model = DummyClassifier(strategy='uniform', random_state=12345)
dummy_model.fit(X_train, y_train)
f1_score(y_test, dummy_model.predict(X_test))

0.16900237035804883

## Выводы

- В ходе знакомства и предобработки данных были сделаны следующие выводы и наблюдения:
    - данные импортированы с удалением лишнего индекса при чтении
    - удалены стоп-слова
    - явных дубликатов не обнаружено
    - в данных наблюдается большой дисбаланс классов
    - данные поделены на две выборки: обучающую и тестовую
    - для улучшения метрики качества была применена техника *downsampling*, но она положительных результатов не дала
- На этапе обучения были подобраны параметры и обучены модели со следующими результатами:
    - При обучении Логистической Регрессии получено значение *f1=0.753* за 5min 37s обучения
    - При обучении Случайного Леса получено значение *f1=0.41* за 5min 6s обучения
    - При обучении CatBoost получено значение *f1=0.9* за 18min 20s обучения
    - С заданием справились две модели, но обучение ***LogisticRegression*** проходит значительно быстрее
- На тестовой выборке данных выбранная модель показала результат удовлетворяющий поставленной задаче: **0.759>0.75**
    - Для проверки адекватности модели точность её предсказаний была сопоставлена с точностью предсказаний *Dummy* модели. Результат выбранной модели значительно лучше результата *Dummy* модели

<br>Общее заключение: для выполнения поставленной задачи заказчика подходит модель ***LogisticRegression***